<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ufidon/nlp/blob/main/08.llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ufidon/nlp/blob/main/08.llms.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>
<br>

# Large Language Models

📝 SALP chapter 10

## 🤔 How do we learn?
- Fluent speakers' vast vocabulary is key for language comprehension and production, aiding in studying knowledge acquisition.

- Adult vocabulary estimates range from 30,000 to 100,000 words, mostly learned early through spoken interactions.

- Children need to learn 7-10 words daily to reach adult vocabulary levels by age 20, a consistent rate across studies.

- Reading drives vocabulary growth, with children learning words faster than they encounter them in texts.

- The distributional hypothesis suggests word meanings are learned from co-occurrences in text, with real-world interactions enhancing this.

## 🤔 Could machines learn similarly?
- Pretraining large language models (LLMs) involves learning language and world knowledge from vast text data, enabling them to excel in various natural language tasks.

- LLMs have transformed tasks like summarization, translation, question answering, and chatbots by using the knowledge gained during pretraining.

- LLMs are often `autoregressive or casual`, predicting the next word from previous ones in text flowing sequence (mostly, left-to-right) during training.

- Text generation with LLMs is central to generative AI, which includes text, code, and image generation, using specific algorithms like greedy decoding and sampling.

- Almost any NLP task, such as summarization, can be framed as `word prediction` in LLMs, demonstrating their versatility.

## Overview of LLMs
- **Architecture**: Large Language Models (LLMs) are deep neural networks, often based on transformer architectures, that can process and generate human-like text by learning patterns in massive datasets.

- **Training**: LLMs are trained on vast amounts of text data from the internet, books, and other sources. This enables them to understand grammar, facts, and relationships between words to perform a variety of natural language tasks.

- **Scale**: These models are "large" due to their immense size, often containing billions or even trillions of parameters (weights and biases), which allows them to capture nuanced information and context in text.

- **Generalization**: LLMs can generalize across a wide range of language tasks like text completion, translation, summarization, and answering questions without task-specific training, relying on their broad training data.

- **Adaptability**: LLMs can be fine-tuned for specific applications, such as customer support, writing assistants, or domain-specific text generation, making them versatile across industries and tasks.

## LLMs Conditional Text Generation
- LLMs generate text token-by-token, using both the `input prompt` and `previously generated tokens`.
  - Long context windows (thousands of tokens) make transformers effective for this task.
- 🍎 Text completion – LLMs predict the next word based on context, leading to coherent outputs.

![Left-to-right (also called autoregressive) text completion](./images/llm/lrtextcomp.png)


### **Word Prediction for NLP Tasks**
- Many practical NLP tasks can be cast as `word prediction`.
- **Sentiment Analysis**: Predict sentiment by comparing probabilities of words like "positive" vs. "negative."
  - e.g. The sentiment analysis of `I like NLP` can be cast to
    - P(positive | The sentiment of `I like NLP` is:), and
    - P(negative | The sentiment of `I like NLP` is:)
- **Question Answering**: Predict the next word after a question to generate factual answers.
  - e.g: "Who wrote *The Origin of Species*?" → Answer: "Charles Darwin." can be cast to
    - P(w|Q: Who wrote the book ‘‘The Origin of Species"?  A:) over all possible next words
      - It is very likely we get `Charles`, add it to the context and continue the prediction
    - P(w|Q: Who wrote the book ‘‘The Origin of Species"?  A: Charles)
      - then it is very likely we get `Darwin`
- **Summarization**: LLMs generate summaries using prompt like long articles appended with `tl;dr` to condense it.
  - Transformers handle large context windows, using the entire article and generated text to produce concise summaries.

![Summarization with large language models using the tl;dr token and context-based autore-
gressive generation](./images/llm/texsum.png)

### **Decoding Strategies in LLMs**
- Choosing a word to generate based on the model’s probabilities is called `decoding`. There are 3 popular decoding strategies:
- **Greedy decoding**: Always selects the most probable next word $\hat{w}_t$ from the vocabulary $V$, but results in repetitive, generic text.
  - $\hat{w}_t = \arg \max_{w∈V} P(w|𝐰_{<t})$
  - Extremely predictable, identical contexts result in same output.
- **Beam search**: Extension of greedy decoding, works well for highly constrained tasks like machine translation.
  - It is expected generating a text in one language conditioned on a very specific text in another language.
- **Sampling methods**: repeatedly randomly samples words according to their probability
until a pre-determined length is reached or the end-of-sentence token is selected. 
  - Introduce diversity by generating less predictable outputs, improving text variation over greedy decoding.
  - It is the most common method for decoding in LLMs with a bit of `generalization`:
    - Conditioned on `prompts and previous selections`, words are sampled based on their conditional probabilities determined by a transformer language model.
    - Three popular sampling schemes: `random` sampling, `top-k` sampling, `nucleus or top-p` sampling, and `temperature` sampling

### Random Sampling 
- Generates a sequence of words $𝐖 = w_1, w_2, \cdots, w_N$, until the end-of-sequence token is hit:

---

```
i ← 1
wᵢ ∼ p(w)
while\ wᵢ != EOS
  i ← i+1   
  wᵢ ∼ p(wᵢ | $𝐰_{<i}$)
```
  - `x ∼ p(x)`: choose x by sampling from the distribution p(x)

---

- Random sampling may generate `strange or incoherent sentences` due to the large amount of low-probability words.
- Alternative sampling methods reduce the chance of selecting unlikely words.
  - by trading off between quality (favoring more probable words) and diversity (including middle-probability words for creativity).
- High-probability words lead to coherent but repetitive text, while middle-probability words enhance creativity at the cost of coherence.

### Top-k Sampling
- Top-k sampling generalizes greedy decoding by `selecting from the top k most likely words` instead of the single most probable word.
- At `each word generation`, the vocabulary is `truncated` to the top k words based on their likelihood, and the distribution is `renormalized`.
- A word is randomly sampled from these k words based on their `renormalized probabilities`.
- When `k = 1`, top-k sampling behaves the `same as greedy` decoding.
- Larger k values introduce more diverse text while maintaining quality by selecting words that are still sufficiently probable.

### Nucleus or top-p sampling
- Top-k sampling has a fixed k, but `word probability distributions vary by context`, making it less adaptable.
- `Top-p` sampling (nucleus sampling) selects words based on `covering a fixed p percent of the probability mass` instead of a fixed number of words.
- This approach aims to remove unlikely words while being more flexible across different contexts.
- Top-p sampling `dynamically adjusts the pool of candidate words`, ensuring better adaptability to varying probability distributions.
- Given a distribution $P(w_t |𝐰_{<t} )$, the top-p vocabulary $V^{(p)}$ is the smallest set of words such that 
  - $\displaystyle \sum_{w∈V^{(p)}}  P(w|𝐰_{<t} ) ≥ p$

### Temperature sampling
- Temperature sampling `reshapes the probability distribution` instead of truncating it, `adjusting word probabilities` based on a `temperature parameter` $τ$.
- In `low-temperature` sampling $τ ∈ (0,1]$, the probabilities of common words increase, making the distribution more `focused on high-probability` words.
  - The logits are divided by τ before being passed through softmax, enhancing the probability of the most likely words.
  - As τ approaches 0, the model becomes more "greedy," favoring the most probable word, while τ close to 1 leaves the distribution mostly unchanged.
- `High-temperature` sampling $τ > 1$ flattens the distribution, encouraging `more exploration and diversity` in word selection.

## Pretraining LLMs with Self-Supervision
- A transformer language model is trained using `self-supervision`, predicting the next word in a text sequence `without needing additional labels`.
- The model minimizes prediction errors by using `cross-entropy loss`, which measures the difference between the predicted ${\hat{𝐲}}_t$ and actual ${𝐲}_t$ probability distributions.
  - $\displaystyle L_{CE} = -\sum_{w\in V} {𝐲}_t[w]\log {\hat{𝐲}}_t[w]$
  - ${𝐲}_t$: a `one-hot vector` corresponding to the vocabulary where the entry for the actual next word is 1, and all the other entries are 0.
- Cross-entropy loss is simplified by focusing on the probability assigned to the correct next word in the sequence.
  - $\displaystyle L_{CE}({\hat{𝐲}}_t, {𝐲}_t) = -\log {\hat{𝐲}}_t[w_{t+1}]$
- At each time step, the model computes a probability distribution for the next word based on the correct input sequence.
- `Teacher forcing` is used, where the correct sequence of tokens is always fed to the model for prediction, rather than using its previous predictions.


![Training a transformer as a language model](./images/llm/trainllm.png)

- The `average cross-entropy loss over the entire sequence` is calculated, and weights are adjusted via gradient descent to minimize this loss.
- Unlike RNNs, transformers `process each item in the sequence in parallel`, as there’s no recurrence in hidden state calculations.
- Large models `fill the full context window` with text, packing multiple documents with special end-of-text tokens if necessary.
- Batch sizes for gradient descent are typically large, with GPT-3 models using up to 3.2 million tokens per batch.

### Training corpora for large language models
- Large language models are primarily trained on `web-scraped text`, supplemented by `curated data`.
- These massive corpora often include natural examples useful for NLP tasks like question-answer pairs, translations, and document summaries.
- [Common Crawl](https://commoncrawl.org/), a widely used web snapshot source, provides billions of webpages for training, with filtered versions like the [Colossal Clean Crawled Corpus (C4) corpus](https://huggingface.co/datasets/gsarti/clean_mc4_it) containing 156 billion English tokens.
- Other significant sources include Wikipedia, books, and specialized datasets like [The Pile](https://pile.eleuther.ai/) (825 GB) and [Dolma (three trillion tokens)](https://allenai.github.io/dolma/).
- These datasets consist of web text, academic papers, code, books, encyclopedias, and social media content.

### Filtering for quality and safety
- Pretraining data from the web is filtered for quality using classifiers that `favor high-quality sources` like Wikipedia and books, while `removing low-quality`, PII(Personal Identifiable Information)-heavy, or adult content.
- Quality filters also `remove frequent template` text and `duplicate` documents.
- Safety filtering focuses on detecting and removing toxic content, but toxicity classifiers can misidentify non-toxic data, especially from minority dialects.
- While safety filtering reduces toxicity, it also makes models less effective at detecting toxicity, presenting an ongoing challenge in improving safety measures.
- Using large datasets scraped from the web poses `ethical and legal questions`:
  - **Copyright concerns:** Many datasets used for training language models include copyrighted material, raising questions about whether using such content is permissible, especially when models generate competing texts.
  - **Data consent:** Website owners can opt out of being crawled for training data, but the legal implications of these restrictions, particularly with the rise in such opt-outs, remain unclear.
  - **Privacy issues:** Web datasets often contain private information, like phone numbers and IP addresses, and current filters may not fully eliminate this data, posing privacy risks.

### Finetuning
- **Finetuning** refers to the process of adapting a pretrained language model to a specific domain, task, or language by continuing its training on new, relevant data, updating some or all of its parameters. 

![Pretraining and finetuning](./images/llm/prefine.png)

- There are four types of finetuing:

- **Full finetuning (continued training):** All model parameters are retrained on new domain-specific data, using the same methods as pretraining. This approach is slow and resource-intensive, especially for large models.
- **Parameter-efficient finetuning (PEFT):** Only a subset of parameters are updated, while the rest are frozen. This method is more efficient and less expensive for large models.
- **Task-specific finetuning:** Used primarily with masked language models, this approach adds a classification head to the model. The head is trained on task-specific data (e.g., sentiment classification), while the rest of the model remains frozen.
- **Supervised finetuning (SFT):** Often used for instruction finetuning, this method involves training the model to follow text prompts using a hand-curated dataset of questions, commands, and desired responses.

- **Post-training** refers to the broader set of processes that occur after pretraining, including various types of finetuning and model adaptations, to optimize performance for specific applications.

## Evaluating LLMs with Perplexity
- `Perplexity` is a measure of how well a language model predicts unseen text, with lower perplexity indicating better performance.
- Perplexity is the inverse probability of a test set of $n$ tokens $w_{1:n}$, normalized by the test set length $n$:  
  $$
  \text{Perplexity}_{\theta}(w_{1:n}) = P_{\theta}(w_{1:n})^{-\frac{1}{n}} = \sqrt[n]{\frac{1}{P_{\theta}(w_{1:n})}}
  $$
- Using the chain rule, perplexity can be expanded as:  
  $$
  \text{Perplexity}_{\theta}(w_{1:n}) = \sqrt[n]{\prod_{i=1}^{n} \frac{1}{P_{\theta}(w_i|w_{<i})}}
  $$
- Perplexity comparisons between models are only reliable when the same tokenization algorithm is used, as it is sensitive to tokenization differences.

### Evaluating LLMs with other factors
- Beyond perplexity, language models are evaluated based on their `accuracy` in downstream tasks like machine translation, summarization, and question answering, using `task-specific metrics`.
- Other evaluation factors include model size, training and inference speed, memory usage, and energy consumption, as these affect computational efficiency and environmental impact.
- `Fairness` is an important metric, assessing bias in models related to gender, race, and demographic performance. Benchmarks like [StereoSet](https://paperswithcode.com/dataset/stereoset), [BBQ (Bias Benchmark for QA)](https://paperswithcode.com/dataset/bbq) and [RealToxicityPrompts](https://huggingface.co/datasets/allenai/real-toxicity-prompts) help evaluate these biases.
- `Leaderboards` like [Dynabench](https://dynabench.org/) and `evaluation protocols` like  [Holistic Evaluation of Language Models (HELM)](https://github.com/stanford-crfm/helm) are used to rank models and standardize evaluation across various tasks.

## Dealing with Scale
- LLMs like Meta's Llama 3.1 with 405 billion parameters, require significant resources. 
- Research focuses on understanding `scaling laws` and implementing them efficiently with limited resources. 
- Key techniques include `KV caching` and `parameter-efficient fine-tuning (PEFT)`.

### Scaling laws
- LLMs' performance is mainly determined by model size (N), dataset size (D), and compute budget (C), following [scaling laws](https://github.com/shehper/scaling_laws) that describe `performance improvement through added parameters, data, or compute`.
- The performance (loss, L) of a model scales as a `power-law` with respect to these factors:
   - $L(N) = \left( \frac{N_c}{N} \right)^{\alpha_N}$
   - $L(D) = \left( \frac{D_c}{D} \right)^{\alpha_D}$
   - $L(C) = \left( \frac{C_c}{C} \right)^{\alpha_C}$
   - The values of $N_c , D_c , C_c, α_N , α_D$, and $α_C$ depend on the exact transformer architecture, tokenization, and vocabulary size.
- The number of non-embedding parameters $N$ can be approximated as 
  - $N \approx 2d n_{\text{layer}}(2d_{\text{attn}}+d_{\text{ff}}) ≈ 12n_{\text{layer}}d^2$
    - assuming $d_{\text{attn}} = \dfrac{d_{\text{ff}}}{4} = d$ in transformer.
  - 🍎  GPT-3, with n = 96 layers and dimensionality d = 12288, has 12 × 96 × 12288² ≈ 175 billion parameters.

### KV Cache
- During training, the attention vector is efficiently computed using two matrix multiplications:  
  - $\displaystyle A = \text{softmax} \left( \frac{QK^T}{\sqrt{d_k}} \right) V$

![Parts of the attention computation](./images/llm/kvcache.png)

- In inference, tokens are generated one at a time, and to avoid recomputing key and value vectors for prior tokens, the KV cache is used to store them, ($k_{1:3}, v_{1:3}$) above, for future retrieval.

### Parameter Efficient Fine Tuning (PEFT)
- Fine-tuning LLMs is challenging due to the high computational cost of updating numerous parameters during backpropagation.
- PEFT techniques, like `LoRA (Low-Rank Adaptation)`, address this by freezing most parameters and only updating a smaller subset, reducing resource requirements.
- [LoRA](https://github.com/microsoft/LoRA) focuses on transformer layers that perform matrix multiplication, replacing full matrix updates with low-rank approximations using two smaller matrices, 𝐀 and 𝐁, where $r ≪ \min(d, k)$.
  - Replace ${𝐖+Δ𝐖}$ by ${𝐖+𝐀𝐁}$
- The forward pass is modified from ${𝐡 = 𝐱𝐖}$ to ${𝐡 = 𝐱𝐖 + 𝐱𝐀𝐁}$, reducing hardware demands while maintaining inference speed, as the updates are the same size as the original matrix.

![The intuition of LoRA](./images/llm/lora.png)

- LoRA allows modular updates for different domains by adding or subtracting low-rank modules during inference, with many variants applied to attention layers like ${𝐖_𝐐, 𝐖_𝐊, 𝐖_𝐕, 𝐖_𝐎}$.

## Potential Harms from Language Models
- Large language models pose risks like `hallucination (false information generation), toxic language, and demographic biases`, especially in downstream tasks like text generation, translation, and assistive technologies.
- `Hallucination` occurs because models generate coherent but potentially incorrect text, which is problematic in applications where factual accuracy is crucial.
- Language models can produce `toxic language, stereotypes, and biased content` due to the biases present in training data, which often includes toxic or skewed perspectives from specific demographic groups.
- Pretraining on large web-scraped corpora can lead to issues like `privacy leaks` (e.g., extracting personal information) and `copyright violations`, especially when sensitive data like health records are involved.
- Language models can be exploited by malicious actors for generating harmful content (e.g., misinformation, phishing), highlighting the need for mitigation strategies, transparent training data documentation (datasheets or model cards), and regulations.